In [1]:
using JuMP
using GLPK
using ECOS
using MAT
using SparseArrays
using LinearAlgebra
using MathOptInterface
const MOI = MathOptInterface
using DelimitedFiles
using COSMO
using CSV
using DataFrames
using DelimitedFiles
using Suppressor

In [2]:
vars = matread("../R5T3.mat")
S = vars["S"]
U = vars["U"]
L = vars["L"]
K = vars["K"]
n,d = size(S)
print(size(S))
print(size(L))

n,d = size(S) # n=10 d=100
d,k = size(L) # k = 5

print("done")

target = size(L)[2] - K ;

(15638, 28301)(28301, 200)done

In [3]:
X = zeros(size(L)[1],size(L)[2]);


In [4]:
function f_zero_rows(X)
    zero_rows = 0
    for i in 1:size(X)[1]
        zero_rows += Int16( (sum( X[i,:] .>= 2e-5 ) + sum( X[i,:] .<= -2e-5 )) == 0)
    end
    return zero_rows
end;

In [5]:
function f_zero_cols(Z)
    zero_cols = 0
    for i in 1:size(Z)[2]
        zero_cols += Int16( (sum( Z[:,i] .> 2e-5 ) + sum( Z[:,i] .< -2e-5 )) == 0)
    end
    return zero_cols
end;

In [ ]:
n,d = size(S)
lambda = 1
zero_cols = 0
X_c = copy(X)
@suppress_err begin
    
    while(zero_cols < target)
        for j in 1:size(X)[2]
            model = Model(ECOS.Optimizer)
            @variable(model, L[i,j] <= x[i = 1:d] <= U[i,j],start=X[i,j])
            @variable(model, y_abs[i = 1:n])
            @constraint(model,y_abs .>= S*x)
            @constraint(model,y_abs .>= -S*x)
            @variable(model, x_abs[1:d])
            @constraint(model, x_abs .>= x)
            @constraint(model, x_abs .>= -x)
            @objective(model, Min, sum(y_abs[i] for i in 1:n) + (1/lambda)*sum(x_abs[i] for i in 1:d));
            set_silent(model)
            optimize!(model)
            x = value.(x);
            X[:,j] = x;
            print(" ,",j)
        end
        print("\n")
        println("zero cols: ", f_zero_cols(S*X))
        println("zero rows: ", f_zero_rows(X))
        println("lambda: ", lambda)
        
        if(zero_cols >= target)
            break
        else
            X_c = copy(X)
        end
        lambda = sqrt(lambda * (target-zero_cols))
        println("next lambda: ", lambda)
        println("-------------------------------")
    end
end

 ,1 ,2 ,3 ,4 ,5 ,6

In [ ]:
println(sum(vec(X_c) .> vec(U)+ones(size(vec(U))[1])*2e-5));
println(sum(vec(X_c) .< vec(L)-ones(size(vec(U))[1])*2e-5));

In [ ]:
writedlm("output.txt", Matrix(reshape(value.(X_c),(d,k))), ',')